# Geocoding City and State

### Setup and Imports

In [1]:
!pip install geopy

In [2]:
!pip install mysql-connector-python

In [3]:
# Imports and display options

import pandas as pd
import requests
import time

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

pd.options.display.max_rows = 1000 
pd.options.display.max_columns = 1000
pd.options.display.max_colwidth = 1000

### Load Data

In [4]:
# Using just the first 10 rows for this example
df = pd.read_csv('unique_city_state_example.csv', nrows=10)

print(df.shape)
df.head()

(10, 2)


,city,state
0,Abbotsford,BC
1,Aberdeen,MD
2,Aberdeen,SD
3,Abilene,TX
4,Accord,NY


### Create State Name

To have a bit more information to work with, map the state abbreviation to its full name.

In [5]:
# United States of America Python Dictionary to translate States,
# Districts & Territories to Two-Letter codes and vice versa.
#
# Canonical URL: https://gist.github.com/rogerallen/1583593
#
# Dedicated to the public domain.  To the extent possible under law,
# Roger Allen has waived all copyright and related or neighboring
# rights to this code.  Data originally from Wikipedia at the url:
# https://en.wikipedia.org/wiki/ISO_3166-2:US
#
# Automatically Generated 2021-09-11 18:04:36 via Jupyter Notebook from
# https://gist.github.com/rogerallen/d75440e8e5ea4762374dfd5c1ddf84e0 

# Canada has been added to Roger Allen's code to accommodate Ryadd's locations

us_state_canada_province_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
    "US Virgin Islands": "VI",
    "Alberta": "AB",
    "British Columbia": "BC",
    "Manitoba": "MB",
    "New Brunswick": "NB",
    "Newfoundland and Labrador": "NL",
    "Northwest Territories": "NT",
    "Nova Scotia": "NS",
    "Nunavut": "NU",
    "Ontario": "ON",
    "Prince Edward Island": "PE",
    "Quebec": "QC",
    "Saskatchewan": "SK",
    "Yukon": "YT",
}
    
# invert the dictionary
abbrev_us_state_canada_province = dict(map(reversed, us_state_canada_province_to_abbrev.items()))

In [6]:
# Leverage the inverted dictionary to create full state name
df['state_name'] = df['state'].apply(lambda x: abbrev_us_state_canada_province[x])
df.head(5)

,city,state,state_name
0,Abbotsford,BC,British Columbia
1,Aberdeen,MD,Maryland
2,Aberdeen,SD,South Dakota
3,Abilene,TX,Texas
4,Accord,NY,New York


### Geocoding

In [7]:
# The below will return a new dataframe with the coordinates for a city, state

# To avoid excessive calls, create a dataframe that is the unique combinations of city and state name
unique_city_states = (df
                      .groupby(['city', 
                                'state_name'])
                      .size()
                      .reset_index(name='Freq')
                     )

# Create name as the combination of city and state (comma separated)
unique_city_states['name'] = unique_city_states['city'] + ", " + unique_city_states['state_name']

# Printing output for reference
print(unique_city_states.shape)
print(unique_city_states.head(1))

# Parameters for geocoding
geolocator = Nominatim(user_agent="usertest98")
geocode = RateLimiter(geolocator.geocode, 
                      min_delay_seconds=2, 
                      max_retries=0)

# To avoid hitting API rate limits, function to sleep between calls
def geocode_with_delay(location_str):
    loc = geocode(location_str)
    time.sleep(1)
    return tuple(loc.point) if loc else None


# Create the lat/lon coordinates
unique_city_states['point'] = unique_city_states['name'].apply(geocode_with_delay)

unique_city_states.head(5)    

(10, 4)
         city        state_name  Freq                          name
0  Abbotsford  British Columbia     1  Abbotsford, British Columbia


,city,state_name,Freq,name,point
0,Abbotsford,British Columbia,1,"Abbotsford, British Columbia","(49.0521162, -122.329479, 0.0)"
1,Aberdeen,Maryland,1,"Aberdeen, Maryland","(39.5120347, -76.1643289, 0.0)"
2,Aberdeen,South Dakota,1,"Aberdeen, South Dakota","(45.4649805, -98.487813, 0.0)"
3,Abilene,Texas,1,"Abilene, Texas","(32.44645, -99.7475905, 0.0)"
4,Accord,New York,1,"Accord, New York","(41.7979985, -74.22923853406763, 0.0)"
